# Introducción a Pandas

## Importación y exportación

Podemos importar datos a DataFrames de Pandas de diferentes orígenes y formatos, entre ellos:

* De CSV: con [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)
* De Excel: con [`read_excel`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html)
* De base de datos: con [`read_sql`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html)

Como ejemplo, vamos a importar unos datos que tenemos en CSV con precios de alquileres en los distritos de Madrid:

In [1]:
import pandas as pd

alquiler = pd.read_csv('dat/alquiler-madrid-distritos.csv', index_col=False)
alquiler.head()

,distrito,ano,quarter,precio
0,Arganzuela,2007,2,13.066587
1,Barajas,2007,2,11.199855
2,Carabanchel,2007,2,11.127661
3,Centro,2007,2,17.746404
4,Chamartín,2007,2,14.386480


También podemos exportar esos datos a CSV haciendo:

In [2]:
alquiler.to_csv('alquiler.csv')

Pandas es una librería de Python utilizada para tratar datos en forma de tabla. Nos permite importar, exportar y hacer las operaciones habituales que nos permiten otras herramientas como Excel o el lenguaje SQL

#### Ejercicio 

Comprueba que se ha guardado el fichero correctamente. Antes de nada, tendrás que ubicarlo en tu disco duro. Debería estar en el directorio en el que se guardan los _notebooks_.

#### Nota

Si te interesa saber cómo lanzar consultas a una base de datos usando SQL, puedes leer [este tutorial](https://www.pybonacci.org/2015/03/17/pandas-como-interfaz-sql/) (y los enlaces que contiene).

#### Ejercicio

Usa la función [`read_excel`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html) para importar a Python alguna tabla de tu interés

In [3]:
import pandas as pd

taula_amics = pd.read_csv('/home/lluismo/Documents/curs-python-dades/notebooks/dat/amics.csv', index_col=False)
taula_amics.head()

,Amics,Excursions,CP
0,Vicens,1,8021
1,Nach,2,8022
2,Pau,3,8023
3,Jordi,4,8024
4,Nonell,5,8025


## Estructura básica e inspección

Las tablas en `pandas` son objetos de la clase `DataFrames`. Un `DataFrame` consta de dos partes: un índice y los datos propiamente dichos. Las columnas de los datos son de la clase `Series`.

Para consultar las columnas de un `DataFrame`, accedemos a la propiedad `columns`.

In [4]:
alquiler.columns

Index(['distrito', 'ano', 'quarter', 'precio'], dtype='object')

Si además queremos saber el tipo del dato, accedemos a la propiedad `dtypes`.

*Nota*: las cadenas de texto se marcan como `object` dentro de un DataFrame

In [5]:
alquiler.dtypes

distrito     object
ano           int64
quarter       int64
precio      float64
dtype: object

Cada `DataFrame` tiene un índice. Si no lo hemos especificado, será un incremental sin relación con nuestros datos. El uso de índices está recomendado cuando tratamos con datos grandes, ya que permite acceder a las filas por _hash_ en lugar de tener que iterar por todas ellas para encontrar el valor que se busca. Los índices también son importantes a la hora de realizar agregaciones y cruces entre tablas.

Para consultar cuál es el índice de un DataFrame, accedemos a la propiedad `index`.

In [6]:
alquiler.index

RangeIndex(start=0, stop=840, step=1)

Podemos alterarlo con `set_index`. El nuevo índice puede ser una o varias columnas.

In [7]:
alquiler_nuevo_indice = alquiler.set_index(['distrito', 'ano', 'quarter'])

Una forma rápida de echar un vistazo a los datos es consultas las primeras o últimas filas del DataFrame, con las funciones `head` y `tail`.

In [8]:
alquiler.head()

,distrito,ano,quarter,precio
0,Arganzuela,2007,2,13.066587
1,Barajas,2007,2,11.199855
2,Carabanchel,2007,2,11.127661
3,Centro,2007,2,17.746404
4,Chamartín,2007,2,14.386480


In [9]:
alquiler.tail()

,distrito,ano,quarter,precio
835,Tetuan,2018,2,15.114558
836,Usera,2018,2,11.533458
837,Vicálvaro,2018,2,9.962139
838,Villa De Vallecas,2018,2,10.915967
839,Villaverde,2018,2,10.427527


Podemos seleccionar un listado de columnas a devolver de la siguiente forma:

In [10]:
alquiler[['distrito', 'precio']].head()

,distrito,precio
0,Arganzuela,13.066587
1,Barajas,11.199855
2,Carabanchel,11.127661
3,Centro,17.746404
4,Chamartín,14.386480


Para conocer el número de filas de una tabla hay varias opciones:

In [11]:
len(alquiler)

840

In [12]:
alquiler.shape

(840, 4)

##### Nota

El índice no forma parte propiamente de los datos:

In [13]:
alquiler_nuevo_indice.shape

(840, 1)

### Ejercicio

* Carga en un dataframe de pandas el csv `dat/alquiler-madrid-municipios.csv` en una variable que se llame `alquiler_municipios`
* Examina las primeras y últimas filas
* Extrae el número de filas y columnas

In [14]:
import pandas as pd
alquiler_municipios = pd.read_csv('dat/alquiler-madrid-municipios.csv', index_col=False)
alquiler_municipios.head()

,municipio,ano,quarter,precio
0,Alcalá De Henares,2009,4,8.561386
1,Alcobendas,2009,4,NaN
2,Alcorcón,2009,4,8.792037
3,Alpedrete,2009,4,NaN
4,Aranjuez,2009,4,7.750625


In [15]:
alquiler_municipios.tail()

,municipio,ano,quarter,precio
1185,Torrelodones,2018,2,10.553534
1186,Tres Cantos,2018,2,10.781348
1187,Valdemoro,2018,2,7.812653
1188,Villanueva De La Cañada,2018,2,11.632276
1189,Villaviciosa De Odón,2018,2,14.050801


In [16]:
alquiler_municipios.shape

(1190, 4)

## Filtro y selección

Hay tres operadores fundamentales para seleccionar filas y columnas: `loc`, `iloc` y `[]`. La diferencia fundamental entre `loc` e `iloc` es que el primero requiere _etiquetas_ y el segundo, índices numéricos (la `i` inicial viene de `integer`).


### Selección por índices numéricos

Para acceder por posición usando índices numéricos, se usa `iloc[]`, como en los siguientes ejemplos:

In [17]:
# por defecto, seleccionamos filas
alquiler_nuevo_indice.iloc[200]

precio    9.901414
Name: (Moratalaz, 2010, 4), dtype: float64

In [18]:
# pero también se pueden seleccionar filas y columnas
# además, usando rangos
alquiler.iloc[3:5, 1:]

,ano,quarter,precio
3,2007,2,17.746404
4,2007,2,14.386480


In [19]:
# índices no consecutivos
# recuerda: en python, se empieza a contar en 0
alquiler.iloc[[1, 2, 4], [0, 3]]

,distrito,precio
1,Barajas,11.199855
2,Carabanchel,11.127661
4,Chamartín,14.386480


In [20]:
# los índices negativos indican que se empieza a contar desde el final
alquiler.iloc[-3:-1]

,distrito,ano,quarter,precio
837,Vicálvaro,2018,2,9.962139
838,Villa De Vallecas,2018,2,10.915967


### Ejercicio

* Muestra las primeras 5 filas usando `iloc`
* Muestra las últimas 5 filas usando `iloc`

In [21]:
alquiler_municipios.iloc[:5] 

,municipio,ano,quarter,precio
0,Alcalá De Henares,2009,4,8.561386
1,Alcobendas,2009,4,NaN
2,Alcorcón,2009,4,8.792037
3,Alpedrete,2009,4,NaN
4,Aranjuez,2009,4,7.750625


In [22]:
alquiler_municipios.iloc[-5:] 

,municipio,ano,quarter,precio
1185,Torrelodones,2018,2,10.553534
1186,Tres Cantos,2018,2,10.781348
1187,Valdemoro,2018,2,7.812653
1188,Villanueva De La Cañada,2018,2,11.632276
1189,Villaviciosa De Odón,2018,2,14.050801


### Selección por etiquetas

Para acceder por _etiquetas_ (es decir, columnas parte del índice), se usa `loc[]`

In [23]:
alquiler_nuevo_indice.loc[('Centro', 2014, 2)]

precio    13.39
Name: (Centro, 2014, 2), dtype: float64

In [24]:
# O un distrito completo
alquiler_nuevo_indice.loc[('Centro')].head()

precio
ano  quarter           
2007 2        17.746404
     4        18.044594
2008 2        17.618608
     4        16.858053
2009 2        15.788713

#### Ejercicio

Muestra sobre `alquiler_nuevo_indice` las filas para distrito `Retiro` y año 2012.

In [25]:
alquiler_nuevo_indice.loc[('Retiro',2012)]

/home/lluismo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,precio
quarter,
1,12.601891
2,12.363832
3,12.167435
4,11.951317


### Selección por condiciones

Para extraer las filas que cumplen una condición, le pasamos al DataFrame una Series de booleanos, o directamente algo que la devuelva.

In [26]:
alquiler[alquiler.distrito == 'Retiro'].head()

,distrito,ano,quarter,precio
13,Retiro,2007,2,13.747522
34,Retiro,2007,4,14.021948
55,Retiro,2008,2,14.240559
76,Retiro,2008,4,13.974290
97,Retiro,2009,2,13.019647


**Nota:** mira cómo en el código anterior hemos seleccionado la columna `distrito` usando un punto, `alquiler.distrito`. Es una alternativa a la notación mediante corchetes, `alquiler['distrito']`.

**Para profundizar:** puedes leer las diferentes ventajas e inconvenientes de la notación punto con respecto a la de corchetes [aquí](https://www.dataschool.io/pandas-dot-notation-vs-brackets/) y decidir cuál prefieres utilizar.

Podemos combinar varias condiciones con `&` (y lógico) y `|` (o lógico)

In [27]:
# No olvides los paréntesis, es importante por prioridad de operandos!

alquiler[(alquiler.distrito == 'Retiro') & (alquiler.ano == 2012)]

,distrito,ano,quarter,precio
307,Retiro,2012,1,12.601891
328,Retiro,2012,2,12.363832
349,Retiro,2012,3,12.167435
370,Retiro,2012,4,11.951317


#### Ejercicio

Extrae los nombres de los distritos cuyo precio por metro cuadrado es superior a 15€ en el año y trimestre más reciente del que tenemos datos (míralo imprimiento las últimas filas de la tabla).

In [28]:
# 1. Imprime las últimas filas de la tabla
alquiler.tail()

,distrito,ano,quarter,precio
835,Tetuan,2018,2,15.114558
836,Usera,2018,2,11.533458
837,Vicálvaro,2018,2,9.962139
838,Villa De Vallecas,2018,2,10.915967
839,Villaverde,2018,2,10.427527


In [29]:
# 2. Haz el filtro. Una vez hecho, saca solo la columna distrito
alquiler[(alquiler.ano ==2018) & (alquiler.quarter==2) & (alquiler.precio>15)]

,distrito,ano,quarter,precio
822,Centro,2018,2,18.812639
823,Chamartín,2018,2,16.606506
824,Chamberí,2018,2,17.749769
829,Moncloa,2018,2,15.189176
832,Retiro,2018,2,15.767861
833,Salamanca,2018,2,19.089243
835,Tetuan,2018,2,15.114558


## Ordenación

Podemos ordenar un DataFrame por una o varias columnas, de forma ascendente o descendente, con [`sort_values`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html)

In [30]:
alquiler.sort_values('distrito', ascending=True).head()

,distrito,ano,quarter,precio
0,Arganzuela,2007,2,13.066587
504,Arganzuela,2014,3,10.774739
483,Arganzuela,2014,2,10.680000
462,Arganzuela,2014,1,10.937478
441,Arganzuela,2013,4,10.885807


In [31]:
alquiler.sort_values(['ano', 'quarter', 'distrito'], ascending=[False, False, True]).head()

,distrito,ano,quarter,precio
819,Arganzuela,2018,2,14.596080
820,Barajas,2018,2,12.348824
821,Carabanchel,2018,2,11.613533
822,Centro,2018,2,18.812639
823,Chamartín,2018,2,16.606506


#### Ejercicio

Extrae de mayor a menor por precio las filas de la tabla para Tetuán a partir del año 2017.

In [32]:
alquiler[alquiler.distrito=='Tetuán']

,distrito,ano,quarter,precio


In [40]:
alquiler[(alquiler.distrito == 'Retiro')&(alquiler.ano>2017)].sort_values(['precio'], ascending=[False])

,distrito,ano,quarter,precio
832,Retiro,2018,2,15.767861
811,Retiro,2018,1,15.653564


## Transformación

Nuevas columnas calculadas, cambio de tipo de dato, eliminar una columna



### Crear una columna calculada

Podemos operar sobre las columnas para crear otras nuevas

In [39]:
# Hago una copia para no modificar el dataframe original
alquiler_2 = alquiler.copy()

alquiler_2['precio_90m'] = alquiler_2.precio * 90
alquiler_2.head()

,distrito,ano,quarter,precio,precio_90m
0,Arganzuela,2007,2,13.066587,1175.992857
1,Barajas,2007,2,11.199855,1007.986923
2,Carabanchel,2007,2,11.127661,1001.489519
3,Centro,2007,2,17.746404,1597.176343
4,Chamartín,2007,2,14.386480,1294.783156


Las operaciones que no se pueden lanzar directamente sobre la `Series` completa, la ejecutamos por elemento utilizando `apply`

In [37]:
# Fíjate bien en la función lambda, es una función en una sola línea
alquiler_2['ano_quarter'] = alquiler_2.apply(lambda fila: str(fila.ano) + 'Q' + str(fila.quarter), axis=1)
alquiler_2.head()

,distrito,ano,quarter,precio,precio_90m,ano_quarter
0,Arganzuela,2007,2,13.066587,1175.992857,2007Q2
1,Barajas,2007,2,11.199855,1007.986923,2007Q2
2,Carabanchel,2007,2,11.127661,1001.489519,2007Q2
3,Centro,2007,2,17.746404,1597.176343,2007Q2
4,Chamartín,2007,2,14.386480,1294.783156,2007Q2


#### Ejercicio

Crea una nueva columna `precio_120m` sobre `alquiler_2` que represente el precio de 120 metros cuadrados, pero utilizando `apply` y una función `lambda`.

In [38]:
alquiler_2['precio_120m'] = alquiler_2.apply(lambda fila:alquiler.precio*120, axis=1)
alquiler_2.head()

ValueError: Wrong number of items passed 840, placement implies 1

### Renombrar una columna

Fíjate en la documentación de `rename` y sus ejemplos, [aquí](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html).

Por ejemplo, para renombrar la columna `precio_90m` a `precio_90_metros` sería:

In [41]:
alquiler_2 = alquiler_2.rename(columns={'precio_90m': 'precio_90_metros'})
alquiler_2.head()

,distrito,ano,quarter,precio,precio_90_metros
0,Arganzuela,2007,2,13.066587,1175.992857
1,Barajas,2007,2,11.199855,1007.986923
2,Carabanchel,2007,2,11.127661,1001.489519
3,Centro,2007,2,17.746404,1597.176343
4,Chamartín,2007,2,14.386480,1294.783156


Fíjate en el ejemplo anterior. En general, las funciones de `pandas` crean un nuevo objeto con el resultado de la operación, pero no modifican el actual. En estas funciones, suele haber un parámetro `inplace` con valor por defecto a `False`. Si lo ponemos a `True`, la operación se realiza sobre el objeto que pasamos por parámetro.

#### La importancia de la nomenclatura

Tener buenos nombres de columnas en un DataFrame es importante. Hará mucho más legible nuestro código si nuestras columnas tienen nombres descriptivos, sin caracteres extraños y separados por `_`.

Unos cuantos ejemplos de malos nombres:

* `col1`, `col2`, ..., `colN`: no sabemos qué es cada cosa.
* `precio euros`, `metros cuadrados`: los espacios dificultan escribir código. Por ejemplo, ya no podremos acceder a las columnas con la notación `dataframe.columna`.
* `año`, `variación`, `precio_€`: los caracteres no-asciii (que no son letras no acentuadas ni números) pueden dar problemas al compartir código (p.e. entre Linux y Windows), al exportar / importar, etc. Es mejor evitarlos.
* `PrecioEuros`, `MetrosCuadrados`: aunque es más sutil, el estándar en Python es escribir en snake_case. Es decir, utilizando minúsculas y usando `_` para separar palabras.

Ejemplo de buenos nombres:

* `distrito`, `precio_euros`, `metros_cuadrados`

### Eliminar una columna

Podemos utilizar [`drop`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html).

Por ejemplo:

In [42]:
alquiler_2 = alquiler_2.drop(columns=['precio_90_metros'])
alquiler_2.head()

,distrito,ano,quarter,precio
0,Arganzuela,2007,2,13.066587
1,Barajas,2007,2,11.199855
2,Carabanchel,2007,2,11.127661
3,Centro,2007,2,17.746404
4,Chamartín,2007,2,14.386480


#### Cambiar el tipo de dato

Vamos a crear un DataFrame muy simple para verlo.

In [43]:
prueba = pd.DataFrame({'precio': ['10.50', '15.35', '22.15']})
prueba

,precio
0,10.50
1,15.35
2,22.15


In [44]:
prueba.dtypes

precio    object
dtype: object

Tenemos un DataFrame con precios, pero estos son cadenas de texto en lugar de números. Esto va a limitar nuestro análisis: no podremos ejecutar operaciones aritméticas, calcular medias, etc.

Es muy habitual que esto pase en el momento de cargar unos datos. Veremos más adelante cómo solucionar problemas al cargar CSVs de habla española, que utilizan la coma como separador decimal en lugar del punto, que es el que entiende Python (y, en general, todos los lenguajes de programación).

Sobre nuestro ejemplo actual `prueba`, podemos usar [`astype`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) para convertir la columna a numérica. En este caso, como es numérica con decimales, usaremos el tipo `float`.

In [45]:
prueba['precio'] = prueba.precio.astype(float)
prueba.dtypes

precio    float64
dtype: object

## Resumen estadístico

Pandas provee una serie de funciones de resumen estadístico que podemos aplicar sobre una columna concreta, o sobre todas las del DataFrame.

Para un resumen para todas las columnas de número de filas, media, desviación estándar, cuartiles, ... usamos [`describe`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html)

In [46]:
alquiler.describe()

,ano,quarter,precio
count,840.00000,840.00000,840.000000
mean,2012.90000,2.52500,11.545779
std,3.08242,1.11842,2.271275
min,2007.00000,1.00000,7.591541
25%,2010.75000,2.00000,9.853159
50%,2013.00000,2.00000,11.132781
75%,2015.25000,4.00000,12.996690
max,2018.00000,4.00000,19.308607


Podemos utilizar también `sum`, `mean`, `std`, `count`, `min`, `max`, ... sobre el DataFrame o una columna en concreto

In [ ]:
alquiler.mean()

In [ ]:
alquiler.precio.max()

#### Ejercicio

Extrae los cuantiles 0.1 y 0.9 del precio para el distrito `Tetuan`.

In [ ]:
alquiler[(alquiler.distrito == 'Retiro')].quantile(0.1)

In [ ]:
alquiler[(alquiler.distrito == 'Retiro')].quantile(0.9)

## Agrupación

De una forma equivalente a como hacemos en SQL, podemos agregar las tablas y sacar resúmenes de los grupos. La operación en pandas se hace en dos fases:

* El `groupby`: donde especificamos la o las columnas por las que agregar
* La aplicación de la función de agregación sobre una o varias columnas

Un resumen usando una función de agregación sobre todas las columnas del DataFrame

In [ ]:
alquiler.groupby('ano').max()

# Atención, fíjate bien en lo que hace esto. Saca el valor máximo de distrito (alfabéticamente),
#  de quarter y precio (numéricamente), pero no representa filas completas
# Es decir, Villaverde en el quarter 4 no tuvo ese precio

Para hacerlo únicamente sobre una columna:

In [ ]:
alquiler.groupby('ano').precio.min()

Para aplicar diferentes resúmenes sobre diferentes columnas

In [47]:
tmp = alquiler.groupby('ano').agg({'precio': 'mean', 'distrito': 'first'})
tmp.head()

,precio,distrito
ano,,
2007,12.679226,Arganzuela
2008,12.963422,Arganzuela
2009,11.900691,Arganzuela
2010,11.608368,Arganzuela
2011,11.461966,Arganzuela


#### Ejercicio

Extrae el precio máximo histórico para cada distrito a partir del 2010

In [ ]:
alquiler[alquiler.ano>2010].groupby('distrito').precio.max()

Puedes ver más información sobre agrupaciones en la [documentación de pandas](https://pandas.pydata.org/pandas-docs/stable/groupby.html). Es especialmente útil la parte sobre transformaciones.

## Cruce

Podemos cruzar dos tablas por una o varias columnas en pandas, de forma equivalente a como hacemos en SQL, con [`merge`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html). También podemos usar los distintos tipos de cruce:

* `inner`: para obtener solamente los registros que crucen en ambas tablas
* `left` o `right`: para mantener los registros de una de las dos tablas, crucen o no con la otra
* `outer`: para manter los registros de ambas tablas, crucen o no

In [48]:
# Aquí, además, un ejemplo de cómo crear un dataframe a partir de un diccionario
df_ejemplo = pd.DataFrame({'distrito':  ['Moratalaz', 'Centro', 'Barajas'],
                           'poblacion': [95000, 150000, 46000]})
df_ejemplo

,distrito,poblacion
0,Moratalaz,95000
1,Centro,150000
2,Barajas,46000


In [ ]:
tmp = df_ejemplo.merge(alquiler, on='distrito')
tmp.head()

In [ ]:
len(tmp)

In [ ]:
tmp = df_ejemplo.merge(alquiler, on='distrito', how='right')
tmp.tail()

In [ ]:
len(tmp)

#### Ejercicio

* Carga en un DataFrame el CSV `dat/venta-madrid-distritos.csv`
* Crúzalo con el que ya tenemos de alquiler. El objetivo es tener, para cada distrito, año y cuatrimestre, tanto el precio de alquiler como el de venta del metro cuadrado. Para saber cómo poner sufijos a las columnas que colisionan en el cruce, mira la documentación de `merge`.
* Extrae los precios medios de venta y alquiler por distrito para todo el histórico
* Extrae, para el año y cuatrimestre más reciente del que haya datos, el distrito donde es más rentable comprar una vivienda para destinarla a alquiler. Es decir, con el ratio precio alquiler / precio venta más alto.

In [49]:
import pandas as pd

venta = pd.read_csv('dat/venta-madrid-distritos.csv', index_col=False)
venta.head()

,distrito,ano,quarter,precio
0,Arganzuela,2001,1,1920.406575
1,Barajas,2001,1,1661.890306
2,Carabanchel,2001,1,1358.557880
3,Centro,2001,1,1857.414416
4,Chamartín,2001,1,2523.688015


In [50]:
alquiler_venta = alquiler.merge(venta, on='distrito', suffixes=('_alquiler', '_venta'))
alquiler_venta.head()

,distrito,ano_alquiler,quarter_alquiler,precio_alquiler,ano_venta,quarter_venta,precio_venta
0,Arganzuela,2007,2,13.066587,2001,1,1920.406575
1,Arganzuela,2007,2,13.066587,2001,2,1989.755278
2,Arganzuela,2007,2,13.066587,2001,3,1974.715394
3,Arganzuela,2007,2,13.066587,2001,4,2027.376614
4,Arganzuela,2007,2,13.066587,2002,1,2150.814674


In [ ]:
alquiler_venta.groupby('distrito').agg({'precio_alquiler': 'mean', 'precio_venta': 'mean'})

In [ ]:
alquiler_venta.tail()

In [53]:
#alquiler_venta.ano_alquiler.dtypes
alquiler_venta[(alquiler_venta.ano_alquiler>2016)| (alquiler_venta.ano_venta>2016)]

,distrito,ano_alquiler,quarter_alquiler,precio_alquiler,ano_venta,quarter_venta,precio_venta
64,Arganzuela,2007,2,13.066587,2017,1,3136.799261
65,Arganzuela,2007,2,13.066587,2017,2,3204.209495
66,Arganzuela,2007,2,13.066587,2017,3,3293.411277
67,Arganzuela,2007,2,13.066587,2017,4,3528.858592
68,Arganzuela,2007,2,13.066587,2018,1,3709.236869
...,...,...,...,...,...,...,...
55995,Villaverde,2018,2,10.427527,2017,2,1406.143111
55996,Villaverde,2018,2,10.427527,2017,3,1446.043053
55997,Villaverde,2018,2,10.427527,2017,4,1439.548577
55998,Villaverde,2018,2,10.427527,2018,1,1522.718612


In [61]:
alquiler_venta['rentable']=alquiler_venta.precio_alquiler/alquiler_venta.precio_venta
alquiler_venta.head()

,distrito,ano_alquiler,quarter_alquiler,precio_alquiler,ano_venta,quarter_venta,precio_venta,rentable
0,Arganzuela,2007,2,13.066587,2001,1,1920.406575,0.006804
1,Arganzuela,2007,2,13.066587,2001,2,1989.755278,0.006567
2,Arganzuela,2007,2,13.066587,2001,3,1974.715394,0.006617
3,Arganzuela,2007,2,13.066587,2001,4,2027.376614,0.006445
4,Arganzuela,2007,2,13.066587,2002,1,2150.814674,0.006075


In [62]:
alquiler_venta[(alquiler_venta.ano_alquiler>2017)& (alquiler_venta.ano_venta>2017)&(alquiler_venta.quarter_alquiler>1)&(alquiler_venta.quarter_venta>1)]

,distrito,ano_alquiler,quarter_alquiler,precio_alquiler,ano_venta,quarter_venta,precio_venta,rentable
2799,Arganzuela,2018,2,14.596080,2018,2,3949.943083,0.003695
5599,Barajas,2018,2,12.348824,2018,2,3225.407929,0.003829
8399,Carabanchel,2018,2,11.613533,2018,2,2058.729335,0.005641
11199,Centro,2018,2,18.812639,2018,2,4987.198876,0.003772
13999,Chamartín,2018,2,16.606506,2018,2,5007.470859,0.003316
16799,Chamberí,2018,2,17.749769,2018,2,5202.006707,0.003412
19599,Ciudad Lineal,2018,2,13.061230,2018,2,2982.579322,0.004379
22399,Fuencarral,2018,2,12.120647,2018,2,3317.630745,0.003653
25199,Hortaleza,2018,2,13.429745,2018,2,3403.176870,0.003946
27999,Latina,2018,2,11.479019,2018,2,2146.217083,0.005348


In [ ]:
#No entenc perquè això no funciona: alquiler_venta[alquiler_venta.ano_alquiler=2018].groupby('distrito')
#alquiler_venta[alquiler_venta.ano_alquiler>2017].groupby('distrito').rentable.max()
#Això tampoc em funciona: alquiler_venta[alquiler_venta.ano_alquiler>2017].groupby('distrito').sort_values(['rentable'], ascending=False)
taula = alquiler_venta[alquiler_venta.ano_alquiler>2017].groupby('distrito').rentable.max()
taula.sort_values(ascending=False)
